# Calculating earnings response coefficients from WRDS

[Earnings response coefficients](https://en.wikipedia.org/wiki/Earnings_response_coefficient) are the stock price reaction to the unexpected part of earnings announcements. This script downloads the necessary data to calculate the coefficients.

The script is a Python translation of the following [guide](http://www.wrds.us/index.php/tutorial/view/estimating_the_earnings_response_coefficient_erc) from WRDS.

Output table:

| var            | description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| rdq            | earnings announcement date                                                            |
| statpers       | day when analyst makes forecast                                                       |
| prccq          | stock price at announcement date                                                      |
| meanest        | mean earnings per share forecast across analysts                                      |
| fpedats        | Forecast Period End Date                                                              |
| actual         | the actual realized earnings per share                                                |
| pends          | earnings announcement date                                                            |
| cumulative_ret | cumulative return (-1 to +1 day around earnings announcement)                         |
| car_dec        | abnormal earnings announcement return (return of same market cap decile as benchmark) |
| car_ff49ind    | abnormal earnings announcement return (Fama French 49 industry return as benchmark)   |
| preAnnRet      | pre-announcement return                                                               |

In [1]:
import pandas as pd
import datetime
import numpy as np
import wrds

In [ ]:
db = wrds.Connection()

In [3]:
# load function 
%run ERC_func.py

In [4]:
db.list_tables(library='comp');
# db.describe_table(library='comp', table='sasdata');

In [5]:
#########################################################
# LOAD DATABASES THAT REMAIN IN THE ENTIRE LOOP ###
#########################################################
# get permno from ccm linking table
query = ("""select gvkey, lpermno, linkdt, linkenddt , linktype, linkprim
        FROM crsp.ccmxpf_linktable """)
dfccm_linktable = db.raw_sql(query)


# get dsenames with ncusip
query = ("""select permno, ncusip, namedt, nameendt
        FROM crsp.dsenames """)
dfdsenames = db.raw_sql(query)
dfdsenames.head()


# ibes tickers
query = str("""SELECT ticker as ibes_ticker, sdates, cusip
                FROM ibes.idsum  """)
dfibesident = db.raw_sql(query)

In [6]:
# download fama french 49 indstry returns as benchmark for abnormal returns
import requests, zipfile, io
r = requests.get("http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/49_Industry_Portfolios_daily_CSV.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("include") # save into include folder

In [91]:
# FF 49 industry & sic code link table
df_sic = pd.read_csv('./include/Siccodes49.csv')
df_sic['unitkey'] = 1 # useful for linking later

# fama french 49 industry returns table
dateparser = lambda x: pd.datetime.strptime(x, '%Y%m%d')
# df_ff49_ret = pd.read_csv('./../../Data/49_Industry_Portfolios_Daily_valweight.csv', skiprows=1, engine='python', index_col=0,
#                 parse_dates=True, date_parser=dateparser) # skipfooter because las line is copywrite string

df_ff49_ret = pd.read_csv('include/49_Industry_Portfolios_Daily.csv', skiprows=9, engine='python', skipfooter=2, parse_dates=True) # skipfooter because las line is copywrite string

# only want first part of csv file
idx_first_empty_row = df_ff49_ret.index[df_ff49_ret.iloc[:,1].isna()][0]
df_ff49_ret = df_ff49_ret.iloc[0:idx_first_empty_row-100,:]

date = df_ff49_ret.iloc[:,0].apply(dateparser)
df_ff49_ret = df_ff49_ret.drop('Unnamed: 0', 1)
df_ff49_ret = df_ff49_ret.replace('%','',regex=True).astype('float')

# in the original file some of the columns have whitespaces, some not... need to remove
df_ff49_ret.columns = df_ff49_ret.columns.str.strip()
df_ff49_ret['date'] = date
# df_ff49_ret['date'] = df_ff49_ret.index.to_pydatetime()

# replace missing
df_ff49_ret.replace(-99.99, np.nan, inplace = True)
df_ff49_ret.replace(-999, np.nan, inplace = True)



df_ff49_ret.head()
# df_ff49_ret.shape

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other,date
0,0.56,-0.07,NaN,-1.39,0.00,-1.44,0.62,-1.27,-0.90,0.12,...,0.15,2.77,-0.02,0.27,0.59,0.47,0.81,0.69,-1.66,1926-07-01
1,0.29,0.06,NaN,0.78,0.70,1.46,0.03,0.00,-0.34,-0.35,...,0.06,0.00,0.01,-0.10,1.04,-0.03,-0.61,-0.45,3.09,1926-07-02
2,-0.33,0.18,NaN,-1.74,0.50,-0.96,-0.06,4.27,-1.20,0.41,...,-0.19,0.77,-0.22,-0.67,0.45,0.31,-0.72,0.04,1.41,1926-07-06
3,3.57,-0.15,NaN,-1.73,-0.12,-0.49,-0.06,-4.10,-0.22,0.16,...,0.18,-3.21,-0.57,-0.70,-1.09,0.21,0.07,-0.40,0.62,1926-07-07
4,0.30,1.12,NaN,-0.15,0.30,-0.49,0.24,0.00,-0.01,0.79,...,0.46,-1.10,-0.38,0.33,0.99,-0.88,-0.36,-0.93,0.98,1926-07-08


In [92]:
# HYPERPARAMETERS for loop
startdate = datetime.date(1983, 1, 1) # when ibes starts:

todaydate = datetime.date.today()

days30 = datetime.timedelta(days=30)
days90 = datetime.timedelta(days=90)

jointype = 'inner'

In [93]:
# test function
mindate = datetime.date(2000, 1, 1)
maxdate = mindate + days90
dferc = get_dferc(db, mindate, maxdate, jointype, dfccm_linktable, dfdsenames, dfibesident, df_sic, df_ff49_ret)
dferc.head()

,gvkey,rdq,datadate,prccq,lpermno,linkdt,linkenddt,linktype,linkprim,ncusip,...,actual,pends,hsiccd,ff49,ff49_str,capn,cumulative_ret,car_dec,car_ff49ind,preAnnRet
0,001004,2000-03-15,2000-02-29,23.7500,54594.0,1972-04-24,2019-05-23,LU,P,00036110,...,0.40,2000-02-29,3724.0,24,Aero,8.0,0.335572,0.332225,-0.011828,0.797674
1,001013,2000-02-17,2000-01-31,65.9375,50906.0,1979-03-16,2010-12-31,LU,P,00088610,...,0.18,2000-01-31,3661.0,37,Chips,10.0,0.199234,0.294757,0.237634,1.122157
2,001034,2000-02-23,1999-12-31,30.7500,65832.0,1984-02-15,2008-12-29,LC,P,02081310,...,0.41,1999-12-31,2834.0,13,Drugs,8.0,-0.106238,-0.114064,-0.076238,0.969858
3,001036,2000-02-02,1999-12-31,19.9375,65453.0,1983-12-06,2001-05-31,LC,C,90991410,...,0.76,1999-12-31,3448.0,17,BldMt,8.0,0.047888,-0.064856,0.035588,1.015385
4,001056,2000-02-02,1999-12-31,10.3750,62500.0,1977-06-30,2007-08-31,LC,P,00776810,...,0.07,1999-12-31,3670.0,37,Chips,6.0,1.260422,1.153237,0.984722,0.970000


In [ ]:
# full loop
i = 0
while True:
    i = i+1
    mindate = startdate + i*days90
    maxdate = mindate + days90
    print('iteration {!s}: mindate is {!s}, maxdate is {!s}'.format(i, mindate, maxdate))
    dfercmore = get_dferc(db, mindate, maxdate, jointype, dfccm_linktable, dfdsenames, dfibesident)
    print('size of new data is {!s}'.format(dfercmore.size))
    dferc = pd.concat([dferc, dfercmore])
    
    if maxdate >= todaydate:
        break

In [ ]:
dferc.set_index(['gvkey', 'datadate'])
dferc = dferc.drop(['linkprim', 'ticker_y','linktype','ibes_ticker', 'ticker_x'], axis=1)
dferc.head()

In [ ]:
dferc_new = dferc.drop_duplicates(subset=['gvkey', 'datadate'], keep='last', inplace=False)

In [ ]:
# dferc_new.to_csv('ERCdata.csv')

In [ ]:
# # save to disk as  HDF5
# store = HDFStore('store.h5')
# store['df'] = df  # save it